In [1]:
from amplpy import AMPL, ampl_notebook

ampl = ampl_notebook(
    modules=["gurobi"],  # modules to install
    license_uuid="default",  # license to use
)  # instantiate AMPL object and register magics

AMPL Development Version 20240404 (MSVC 19.38.33135.0, 64-bit)
Demo license with maintenance expiring 20260131.
Using license file "c:\Users\thuduong\Anaconda3\envs\optima\Lib\site-packages\ampl_module_base\bin\ampl.lic".



### purpose: testing optima with generate all pattern but the max of each FG

In [2]:
## PARAMETER - CASE 0
PARAMS = {"warehouse": "HSC"
          ,"spec_name": "JSH270C-PO" # yeu cau chuan hoa du lieu OP - PO
          ,"thickness": 1.6
          ,"maker" : "CSVC"
          ,"stock_ratio": { #get from app 
                    "limited": None
                    # "default": 2
                    # "user_setting": 4
                }
        #   ,"forecast_scenario": median
          }

#save margin_dict in azure env, margin dictionary by warehouse, margin by range?
margin_dict = { "HSC":
    {#thickness : min_margin
    2.6:8
    ,2 : 8
   , 1.6: 6
   ,3.4:10
    },
    "NQS":
    {
    2.6:10
    ,2 : 8
   , 1.6: 6
    }
}
# GET ALL PARAMS
MIN_MARGIN = margin_dict[PARAMS["warehouse"]][PARAMS["thickness"]]
print(f"MIN_MARGIN:{MIN_MARGIN}")

BOUND_KEY = next(iter(PARAMS['stock_ratio']))
BOUND_VALUE = PARAMS['stock_ratio'][BOUND_KEY]
print(f"BOUND_KEY:{BOUND_KEY}, BOUND_VALUE:{BOUND_VALUE}")

MIN_MARGIN:6
BOUND_KEY:limited, BOUND_VALUE:None


In [3]:
from codes.process_df import filter_by_params
import pandas as pd
import numpy as np

# STEP 1: FILTER DF BY PARAMS
stock_file_path = "data/test_mc_df.xlsx"
finish_file_path = 'data/test_finish_df_Jul2022.xlsx'

finish_df = filter_by_params(finish_file_path, PARAMS)
stock_df = filter_by_params(stock_file_path, PARAMS)

In [4]:
stock_df.drop(stock_df.loc[stock_df['status'].isin(['R:REWIND', 'Z:SEMI MCOIL', 'S:SEMI FINISHED'])].index, inplace=True)

In [5]:
def filter_stock_df(df, stock_id, value_columns):
    status_conditions = df['status'].isin(['R:REWIND'
                            ,'Z:SEMI MCOIL'
                            ,'S:SEMI FINISHED'])
    
    # Sort data according to the priority of FIFO
    sorted_df = df.sort_values(by=['receiving_date', 'weight'], ascending=[True, False])

    # Filter dataframe based on status conditions
    if any(status_conditions):
        sorted_df = sorted_df[sorted_df['status'].isin(['R:REWIND', 'Z:SEMI MCOIL', 'S:SEMI FINISHED'])]
    else:
        sorted_df = sorted_df[sorted_df['status'].isin(['M:RAW MATERIAL'])]

    # Set the index of the DataFrame to 'stock_id'
    sorted_df.set_index(stock_id, inplace=True)
    
    # Select desired columns
    result = sorted_df[value_columns]

    return result

In [ ]:
# STEP 2: PROCESS DATA TO DF THAT CAN USE TO CONVERT TO DICT
stock_key, value_columns = "inventory_id", ['receiving_date','status',"width", "weight"]
filtered_stock_df = filter_stock_df(stock_df, stock_key, value_columns)
filtered_stock_df # chon pool of stock co hang tram tro len giong nhau

In [6]:
def create_bound_need_cut(df, BOUND_VALUE=None):
    """
    Always recalculate with new need cut
    Minus need_cut will always right for DEFAULT & USER SETTING

    IMPROVE: co the chi can 1 cot upper bound
    """
    # Limited case only be in the consideration if NEED_CUT < 0:
    def create_fc_list(x):
        return [f"fc{i}" for i in range(1, x+1)]
    
    if BOUND_VALUE is not None:
        fc_columns = create_fc_list(BOUND_VALUE)

        if BOUND_VALUE <= 3:
            # Default allowing stock after cut < original need cut + X <=3 months forecast
            df['upper_bound_needcut'] = df[fc_columns].sum(axis=1) - df['need_cut']
        else:
            # User setting allowing stock after cut < original need cut + X<=6 months forecast
            df['upper_bound_needcut'] =  df[fc_columns].sum(axis=1) - df['need_cut']
    else:
        df['upper_bound_needcut'] = 0.3 * df['fc1'] - df['need_cut']
    
    def lower_bound(val):
        return min(val, 0)

    # Apply the function to each value in the column
    df['lower_bound_needcut'] = abs(df['need_cut'].apply(lower_bound))
    
    return df

In [7]:
from codes.process_df import create_upper_bound_need_cut

def filter_finish_df(df, finish_id, value_columns, PARAMS):
    
    BOUND_KEY = next(iter(PARAMS['stock_ratio']))
    BOUND_VALUE = PARAMS['stock_ratio'][BOUND_KEY]

    # Create the upper bound for need cut
    filtered_df = create_bound_need_cut(df, BOUND_VALUE)

    # Filter the DataFrame based on the upper bound column
    needcut_df = filtered_df[filtered_df['need_cut'] < 0]

    # Sort DataFrame by 'need_cut'(negative) column in ascending order, 'width' column in descending order
    sorted_df = needcut_df.sort_values(by=['width','need_cut'], ascending=[False,True])

    # Set the index of the DataFrame to 'stock_id'
    sorted_df.set_index(finish_id, inplace=True)

    result = sorted_df[value_columns]

    return result

In [8]:
finish_key, value_columns = "order_id", ["width", "need_cut", 'upper_bound_needcut', 'lower_bound_needcut', "fc1", "fc2", "fc3"]
filtered_finish_df = filter_finish_df(finish_df,finish_key, value_columns, PARAMS)
filtered_finish_df

,width,need_cut,upper_bound_needcut,lower_bound_needcut,fc1,fc2,fc3
order_id,,,,,,,
13,256.0,-2400,3131.236166,2400,2437.453888,1616.803584,1543.312512
12,216.0,-4500,5898.139200,4500,4660.464000,5427.664800,5225.083200
11,155.0,-1200,1662.711000,1200,1542.370000,1663.680000,2235.570000
14,72.5,-2000,2314.174333,2000,1047.247776,1745.412960,909.452016


### Master Problem

In [ ]:
mc_width = 1219
mc_weight = 10000 # filter coil with this weight
finish = filtered_finish_df[["width", 'upper_bound_needcut', 'lower_bound_needcut',]].to_dict(orient='index')
finish

In [ ]:
cuts_dict = {key: 0 for key in finish.keys()}
for f in finish:
    # max number of f that fit on s 
    num_cuts_by_width = int((mc_width -MIN_MARGIN) / finish[f]["width"])
    # max number of f that satisfied the need cut WEIGHT BOUND
    num_cuts_by_weight = int((finish[f]["upper_bound_needcut"] * mc_width ) / (finish[f]["width"]*mc_weight))
    # min of two max will satisfies both
    num_cuts = min(num_cuts_by_width, num_cuts_by_weight)
    # make pattern and add to list of patterns
    if num_cuts > 0:
        cuts_dict[f] = num_cuts

rolls = {
    (f, 1+i): cuts_dict[f] for i, f in enumerate(cuts_dict)
}
rolls

In [ ]:
masterprob = AMPL()
masterprob.eval("reset;")
masterprob.eval(
    """
      reset data;
      param nPatterns integer > 0; #pat
      set PATTERNS = 1..nPatterns; # patterns
      set FINISH; # finished id

      # weight per unit of stock
      param wu > 0;

      param widthF {FINISH} >= 0; #finish width
      param f_lower_demand {FINISH} >= 0; # rolls of width i PO
      # param f_upper_demand {FINISH}; # permitted overrun on any width

      param rolls {FINISH, PATTERNS} >= 0 default 0; # rolls of width i in pattern j

      var b{PATTERNS} binary;  # which stock s is choosen for pattern p
      maximize total_width:
        sum{p in PATTERNS, f in FINISH} b[p] * rolls[f,p] * widthF[f];

      subject to weight_demand {f in FINISH}:
        sum {p in PATTERNS} b[p]*  rolls[f,p] * widthF[f] * wu <= f_upper_demand[f];

    """
    )

#STEP 3: SET VALUE AND PARAMS - Send data to AMPL
# SET INDICES
masterprob.set["FINISH"] = list(finish.keys())
masterprob.param["nPatterns"] = len(finish.keys())
# Send SCALAR
masterprob.param["wu"] = mc_weight/mc_width
# Send VECTOR
masterprob.param["f_upper_demand"] = {f: finish[f]["upper_bound_needcut"] for f in finish.keys()}
# ampl.param["f_lower_demand"] = {f: finish[f]["lower_bound_needcut"] for f in finish.keys()}
masterprob.param["widthF"] = {f: finish[f]["width"] for f in finish.keys()}
# Generate and send initial pattern matrix
masterprob.param["rolls"] = rolls

### Set up for generation loop

In [ ]:
# Set solve options
masterprob.option["solver"] = "gurobi"
masterprob.option["relax_integrality"] = 1

# Create a param for sending AMPL new patterns
masterprob.eval("param newpat {FINISH} integer >= 0;")
new_pattern = ampl.param["newpat"]

### Define the knapsack subproblem

In [ ]:
# Define the knapsack subproblem
subprob = AMPL()
subprob.option["solver"] = "gurobi"
subprob.eval(
    """
    set FINISH;
    param width_s_min_margin integer;
    
    param widthF{FINISH} >= 0; #finish width
    param v{FINISH} >= 0;
    var x{FINISH} integer >= 0;

    maximize profit: sum {f in FINISH} v[f]*x[f];
    subject to feasible_pattern: sum {f in FINISH} widthF[f]*x[f] <= width_s_min_margin;
"""
)
subprob.set["FINISH"] = list(finish.keys())
subprob.param["width_s_min_margin"] = mc_width - MIN_MARGIN
subprob.param["widthF"] = {f: finish[f]["width"] for f in finish.keys()}
values = subprob.param["v"]
kpsolution = subprob.var["x"]
profit = subprob.obj["profit"]

### LOOP

In [ ]:
limits = masterprob.get_constraint("weight_demand")
while True:
    print("Master problem:")
    masterprob.solve()
    print()

    # Retrieve duals & look for new pattern
    # Solve knapsack problem for potential new pattern
    values.set_values(limits.get_values())
    print("Subproblem:")
    subprob.solve()
    print()
    if profit.value() <= 1.000001:
        break

    # Send new pattern to AMPL
    new_pattern.set_values(kpsolution.get_values())
    ampl.eval("let nPatterns := nPatterns + 1;")
    ampl.eval("let {f in FINISH} rolls[f, nPatterns] := newpat[f];")

In [ ]:
# Compute and display integer solution
ampl.option["relax_integrality"] = 0
ampl.solve()

# Retrieve solution
patcut = ampl.var["b"].to_list(skip_index=True)

# Display solution
rolls = ampl.param["rolls"]
npatterns = int(ampl.param["nPatterns"].value())
{p for p in range(npatterns) if patcut[p] > 0}


In [ ]:
rolls

In [ ]:
solution = [
    ([int(rolls[finish[i], p]) for i in range(len(finish))], int(patcut[p]))
    for p in range(npatterns)
    if patcut[p] > 0
]
